# The Query Notebook

To use this notebook, you must first be given permission via your TACC account. Then, you can authenticate to TAPIS and connect to the Neo4j DBMS. 
This notebook has queries to the database storing the architecture for our systems. They will help illuminate how this works.

In [1]:
import os
import py2neo
import pandas
import neo4jupyter
neo4jupyter.init_notebook_mode()
from py2neo import Graph, Node, Relationship
from py2neo import GraphService
import time
import json
import pprint as pp
import requests as r
import datetime
from getpass import getpass
from tapipy.tapis import Tapis
import pytz

def show(res):
    try:
        pp.pprint(res.json())
    except:
        pp.pprint(res.text)

<IPython.core.display.Javascript object>

In [2]:
start = time.time()

# Base URL for Tapis
base_url = "https://icicle.develop.tapis.io"
username = str(input("username"))

# Get Tapis object if it isn't already created.
try:
    t
    if t.base_url == base_url and t.username == username and t.access_token:
        print("Tapis object already exists.")
        if t.access_token.expires_at < datetime.datetime.now(pytz.utc):
            print("Existing Tapis token expired, getting new token.")
            raise
    else:
        print("Creating new Tapis object.")
        raise
except:
    try:
        t = Tapis(base_url = base_url,
                  username = username,
                  password = getpass('password'))
        t.get_tokens()
    except Exception as e:
        print(f"\nBROKEN! timeout: {time.time() - start}\n")
        raise

# V3 Headers
header_dat = {"X-Tapis-token": t.access_token.access_token,
              "Content-Type": "application/json"}

# Service URL
url = f"{base_url}/v3"                   # remote

print(time.time() - start)
print(f"base_url: {base_url}")
print(f"serv_url: {url}")

usernamead1616
password········
5.115863800048828
base_url: https://icicle.develop.tapis.io
serv_url: https://icicle.develop.tapis.io/v3


In [3]:
t.pods.get_pods()

[
 creation_ts: None
 data_attached: []
 data_requests: []
 description: The pod for the REHS final KG
 environment_variables: 
 
 pod_id: finalkg
 pod_template: neo4j
 roles_inherited: []
 roles_required: []
 status: RUNNING
 status_container: 
 message: Pod is running.
 phase: Running
 start_time: 2022-07-29 20:20:36+00:00
 status_requested: ON
 update_ts: None
 url: finalkg.pods.icicle.develop.tapis.io,
 
 creation_ts: None
 data_attached: []
 data_requests: []
 description: Running a copy of the OSU PPod
 environment_variables: 
 
 pod_id: michaelppodtest
 pod_template: neo4j
 roles_inherited: []
 roles_required: []
 status: RUNNING
 status_container: 
 message: Pod is running.
 phase: Running
 start_time: 2022-08-08 17:20:52+00:00
 status_requested: ON
 update_ts: None
 url: michaelppodtest.pods.icicle.develop.tapis.io,
 
 creation_ts: None
 data_attached: []
 data_requests: []
 description: The second pod for the REHS final project.
 environment_variables: 
 
 pod_id: finalkg2
 p

In [4]:
# See pod ID using above jupyter cell
pod_id = str(input("Enter a pod ID. ")).lower()
username, password = t.pods.get_pod_credentials(pod_id=pod_id).user_username, t.pods.get_pod_credentials(pod_id=pod_id).user_password

Enter a pod ID. finalkg


In [5]:
# Connection to neo4j database
graph = Graph(f"bolt+ssc://{pod_id}.pods.icicle.develop.tapis.io:443", auth=(username, password), secure=True, verify=True)

In [6]:
# Draw the architecture diagram!
neo4jupyter.draw(graph, {"TAPIS" : "name", "Local" : "name", "RemoteKG" : "name"})

In [17]:
# Query for all of the nodes. These are the different entities between which connections occur.
query = "MATCH(n) RETURN n"

graph.run(query).to_data_frame()

,n
0,{'name': 'Tapis Pods'}
1,{'name': 'Local Jupyter Notebook'}
2,{'name': 'Tapis Authentication'}
3,{'name': 'Neo4j Knowledge Graph'}


In [19]:
# Run the cell to get a table.
# The left column of the table represents your local notebook (this notebook!), where you send requests from.
# The right column shows the different things that receive the requests in the process of authentication and querying!
query = """
MATCH(n1)-[:REQUEST]-> (n2)
RETURN n1, n2
"""
graph.run(query).to_data_frame()

,n1,n2
0,{'name': 'Local Jupyter Notebook'},{'name': 'Neo4j Knowledge Graph'}
1,{'name': 'Local Jupyter Notebook'},{'name': 'Tapis Pods'}


In [21]:
# Run the cell to get a table.
# Each row has pairs of entities. Each pair has an authentication step, whether it be requesting authentication or verifying it.
query = """
MATCH(n1)-[:AUTHENTICATION]-> (n2)
RETURN n1, n2
"""
graph.run(query).to_data_frame()

,n1,n2
0,{'name': 'Tapis Pods'},{'name': 'Tapis Pods'}
1,{'name': 'Local Jupyter Notebook'},{'name': 'Tapis Authentication'}
2,{'name': 'Tapis Authentication'},{'name': 'Tapis Pods'}
3,{'name': 'Tapis Authentication'},{'name': 'Local Jupyter Notebook'}
